<a href="https://colab.research.google.com/github/nicolastrimborn/RPW_PMJ_Group/blob/master/JupyterNotebook/PMJ_Development_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hot End

The Hotend is controlled with an EL3202 which is intended for DC motor control but ouputs the required PWM and has the required current capacity (2A combined)


## Hardware
 

| Heater | RTD |
|:-------:|:-----:|
|![alt text](https://drive.google.com/uc?id=1mgSImOT2LJb-yuMfArm2hazKT6hKeQA0)        | ![alt text](https://drive.google.com/uc?id=1CI9dN6nvrY5cGpWfGdgr-2OW1aaTft9b)    |
|[Datasheet](https://download.beckhoff.com/download/document/io/ethercat-terminals/el32xxen.pdf)        |  [Datasheet](https://download.beckhoff.com/download/document/io/ethercat-terminals/el73x2en.pdf)|   |
|[RTD Module](https://www.beckhoff.com/EL3202/)      |   [Motor Module](https://www.beckhoff.com/EL7332/)  |


Note:

> If the EL3201/EL3202 is operated with a two-wire connection, the inputs +R and +RL must be bridged by the user.


## Controller Block


Twincat Provides a temperature controller block that was found to work well and was simple to use.  [PID Temp Controller Block](https://infosys.beckhoff.com/english.php?content=../content/1033/tf4110_tc3_temperature_controller/9007199505388299.html&id=)

![alt text](https://drive.google.com/uc?id=1kkOZnxfJZQyU2sGiK8h9wxu0s9Vk8_I1)

* The Controller Supports Auto Tuning of the parameters and employs the inflectional tangent method to obtain the parameters

* Configured with the controller structure - 
[ST_ControllerParameter](https://infosys.beckhoff.com/english.php?content=../content/1033/tf4110_tc3_temperature_controller/9007199505388299.html&id=)

## Scaling

ADC value of EL3202 is stored as INT type.
* LB = -32768
* UB= 32767
* Memory = 16 Bit
* P100 Measurement at Room Temp With Leads = 109 Ohm Module "Value" Reading at RT = 221
* The module has a precision of 0.1 °C per digit
$$Temp(°C)=\frac{ADC_{raw}}{10}$$



## Tuning
Need to set the Tuning paremeters to minimise overshoot
*  In this application We select 
> Slower settling with low overshoot 


![alt text](https://drive.google.com/uc?id=1zZEW2b16kl88dzfWfo_rmYUPFFRc3Eg7)

## State Model
State Model for the operation of Extruder Systems is shown below
![alt text](https://drive.google.com/uc?id=1Qcx-Oep-aHSVyFubSGl_iOsYmScpaWUm)\
[native file](https://drive.google.com/open!id=1KVCdCSzvTIKEIhofl_wVJl3tbSUifrWj)

# Stepper Motor


## Specs
|![alt text](https://drive.google.com/uc?id=1m0vmRihSnLJvtidjZyr2zxn77GmrcX9z) | ![alt text](https://drive.google.com/uc?id=1e1hG3SjZGp06CqxoFPHiZ0RN5QLszW3f)  |
|:-:|:-:|



---



## Calculations

In order to extrude the expected amount of filament it is nessesary to scale the motor step increments to mm of filament extruded.  Thus the following needs to be taken into account:
* Motor Step Angle (0.9Deg)
* Microstepping of the Stepper Driver Module (Locked at 64)
* Gear Ratio and Hobb Diameter


In [5]:
# Motor Parameters
PI = 3.1415
StepAngle = 0.9
MotorSteps = 360/StepAngle
# Driver Parameters
Microstepping = 64
StepsPerRevolution = MotorSteps * Microstepping
# Extruder Properties
GearRatio = 3
HobbDia = 7.3

# Extruder Steps per 1mm of Filament
eStepsPerMM = (MotorSteps * Microstepping * GearRatio)/(HobbDia * PI)
MMPerStep = 1/eStepsPerMM
DistancePerRevolution = StepsPerRevolution * MMPerStep
print("StepsPerRevolution = ", StepsPerRevolution)
print("eStepsPerMM = " ,eStepsPerMM)
print("DistancePerRevolution = " ,DistancePerRevolution)
print("ScalingFactor = " ,MMPerStep)

StepsPerRevolution =  25600.0
eStepsPerMM =  3348.893186441343
DistancePerRevolution =  7.6443166666666675
ScalingFactor =  0.0002986061197916667


### Beckoff tool
![alt text](https://drive.google.com/uc?id=1BjXWi_dFCCp3zlSrKMNpVrAWMXN_VJxx)

### Fine Tuning
Despite the calculation above several factors result in minor difference in the actual length of filament extruded.  Based on this guide the scalefactor was adjusted to ensure accurate extrusion 
* [e_steps per mm guide](https://e3d-online.dozuki.com/Guide/Titan+Marlin+Configuration/35)



In [6]:
desiredExtDist = 100
actualExtDist = 104
overExtrudPercent = actualExtDist/desiredExtDist
MMPerStepTuned = MMPerStep *overExtrudPercent
print("MMPerStepTuned=", MMPerStepTuned)

MMPerStepTuned= 0.00031055036458333334


### Axis_Extruder Parameters
Using the Parameters obtained above the, Encoder was set to "Simulation Encoder" to operate in open loop and velocities and acceleartions were set to avoid skipping steps and driving the extruder too fast

|![alt text](https://drive.google.com/uc?id=1yUlm0TpPt8iUttkU2cDxYavFRvxqTsb0)  |![alt text](https://drive.google.com/uc?id=1qq1dVnbQCe66JC0YspFWy5SCGkXfslBM)|
|:-:|:-:|




# Motor Control

## Drive State Diagram and Transitions

| ![alt text](https://drive.google.com/uc?id=1aonEgO-u-1E-3yUOZEb4QfRzxAKpJB_R)  |![alt text](https://drive.google.com/uc?id=1oRgx5m4o6Z-FkC3u7rePCYn_LI6TQgCj)   |
|:-:|:-:|




## Drive State and Status
The Epos2 Drive is controlled Over CAN via TxPDOs which are configured with the EPOS software over USB the address is determined by node ID
* Status Word: (16 bits) that communicates the states of the drive
* Th_1 Node_ID = 3
* Th_2 Node_ID = 2

---


|![alt text](https://drive.google.com/uc?id=1MAXy9yOu0d468Aeazjn73ph9VA1ugjU_)| ![alt text](https://drive.google.com/uc?id=14Kwsdnyo5GGpvTvo5LNxnTdDEfRBDR3I)
|:-:|:-:|



## Control Word
The Epos2 Drive is controlled Over CAN via RxPDOs which are configured with the EPOS software over USB the address is determined by node ID

* Control Word: (16 bits) that has to be set in the correct 
sequence to ensure the controller is in the correct state to accept the next command

| ![alt text](https://drive.google.com/uc?id=1_4RG8IrNsgCcCCZuwJhPFGzfX31RgPZW)|![alt text](https://drive.google.com/uc?id=10bd0_3ZmasV0W4XHnOh06VRfamIpx5CR) |
|:-:|:-:|

### Example
To Enable the drive from startup for to accept move commands:
1. Drive Boots up in Switched on Disabled mode (Bit 6 set)
2. Need to issue shut command (0x06)
3. Issue Switch On Command (0x07) or Switch On and Enable (0x0F)

### Program and Function Blocks

PRG_AxesEnable Programs handles Enabling and disabling an initialised drive and makes use of the FB_EnableAxis Function block to communicate with the drives and enabling the Axis under TwinCat.

## Encoder Scaling

Motor: [link text](http://www.servovision.com/Motor1/E/157.pdf)\
Encoder: [link text](https://www.maxongroup.com/medias/sys_master/root/8831078924318/2018EN-419.pdf)\
Gearbox [link text](https://www.maxongroup.com/medias/sys_master/root/8831028494366/2018EN-333.pdf)\
Motor and GB Combo:  [link text](https://www.maxongroup.com/maxon/view/service_search?query=359565)

![alt text](https://drive.google.com/uc?id=1hs4-zDvSwsLzOHgHFSBMAGJ7z_bqDv9g)

Similar to the extruder scaling above it was nessesary to scale the encode values to corresponds to the Degrees of rotation of each axis.  Twincat defines a step as quadcounts at this is used to obtain the °/INC Required

![alt text](https://drive.google.com/uc?id=1M8DocFHxlwvJqKjhfo1MMe-ynOPOXW7I)

In [1]:
encoderCounts = 512
RevolutionDeg = 360
GBratio = 128
DriveRatio = 1  # 3:1 (20teeth to 60 teeth when belts are mounted)
SF = (RevolutionDeg/(encoderCounts*4))/GBratio * DriveRatio
print('Scaling factor: ', SF)

Scaling factor:  0.001373291015625


## Motion Control

The EPOS drives are controlled in VelocityMode by providing velocity setpoints and obtaining feedback over CAN directly which is linked directly to Axis Encoder inputs and drive   
* An RxPDO is used set the drives Target Velocity (connected to Axis Drive nDataOut1[0])
* An TxPDO is used for obtaining the feeback.  Position Actual (connected to encoder nDataIn1)
* TwinCAT NCI is responsible for Setpoint generation based on Online jogging or G-Code 

|![alt text](https://drive.google.com/uc?id=1eDjsZqQ4ctjpQCVKU8UBVm8mwylcnJx9)|![alt text](https://drive.google.com/uc?id=1AHMdReAxnlSb2yhJBdxwa8QmpxtHwUX-)   |![alt text](https://drive.google.com/uc?id=1L8-Eh3plynrJqJLsQtKWMXtL6vDciWjJ)   
|:-:|:-:|:-:|


